## RDP - Stone Soup Experiments
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

Continuing experiments - following tutorial #4: Particle Filter

### Got Nowhere
Not quite understanding how this works, or doesn't work. The predictions don't follow the plots.

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all

from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection
from stonesoup.plotter import AnimatedPlotterly

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath

# Tracker Imports
from stonesoup.types.state import GaussianState

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852


## Get Data from BigQuery
* rdp_straight: short, straight flight path
* rdp_extended: longer flight path
* adsb_straight: truth for rdp_straight
* adsb_extended: truth for rdp_extended


## Save/Read  to/from CSV

In [ ]:
from math import pi
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
adsb_file = f'{data_dir}/adsb_straight.csv'
adsb_data = pd.read_csv(adsb_file)
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data = adsb_data.loc[~adsb_data['timestamp'].isna()]
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data['timestamp'] = adsb_data['timestamp'].dt.tz_localize(None)

rdp_file = f'{data_dir}/rdp_straight.csv'
rdp_data = pd.read_csv(rdp_file)
rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)

# Matched Plots
matched_csv = f'{data_dir}/rdp_matched.csv'
rdp_matched = pd.read_csv(matched_csv)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()

print(f'ADSB: {len(adsb_data)}')
print(f'RDP: {len(rdp_data)}')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
target_address = 10537421

file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = matched_data.loc[(matched_data.target_address==target_address) &
                                (matched_data.close_enough==True)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
# Detections
matched_xy = CSVReaderXY(matched_csv)
matched_polar = CSVReaderPolar(matched_csv)

# ADSB
adsb = ADSBTruthReader.multiple_ground_truth_reader([adsb_file])

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]

timestamps = generate_timestamps(start_time, end_time)
plot_all(dets, adsb, start_time, end_time, plot_type='animated')

## From Tutorial #3

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity

from stonesoup.predictor.particle import ParticlePredictor
from stonesoup.resampler.particle import ESSResampler
from stonesoup.updater.particle import ParticleUpdater

from scipy.stats import multivariate_normal

from stonesoup.types.numeric import Probability  # Similar to a float type
from stonesoup.types.state import ParticleState
from stonesoup.types.array import StateVectors
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track

## Set up the particle filter
Analogously to the Kalman family, we create a `ParticlePredictor` and a `ParticleUpdater` which take responsibility for the predict and update steps respectively. These require a `TransitionModel` and `MeasurementModel` as before.
To cope with sample sparsity we also include a resampler, in this instance `SystematicResampler`, which is passed to the updater. It should be noted that there are many resampling schemes, and almost as many choices as to when to undertake resampling. The systematic resampler is described in [#]_, and in what follows below resampling is undertaken at each time-step. More resamplers that are included in Stone Soup are covered in the [Resampler Tutorial](https://stonesoup.readthedocs.io/en/latest/auto_tutorials/sampling/ResamplingTutorial.html#sphx-glr-auto-tutorials-sampling-resamplingtutorial-py)

### Use of Effective Sample Size resampler (ESS)
Resampling removes particles with a low weight and duplicates particles with a high weight. A side effect of this is that additional variance is added. Use of `SystematicResampler` at each time-step means that additional variance is being introduced when it may not necessarily be required. To reduce the additional variance, it may be optimal to resample less frequently.

The Effective Sample Size resampler (`ESSResampler`) compares the variance of the unnormalised weights of the particles to a pre-specified threshold, and only resamples when the variance is greater than this threshold. This threshold is often calculated by the ESS criterion (at time n) given by:
$$
           ESS = \left(\sum_{i=1}^{N} (W_{n}^i)^2\right)^{-1}
$$

### Initialise a prior
To start we create a prior estimate. This is a `ParticleState` which describes the state as a distribution of particles using `StateVectors` and weights. This is sampled from the Gaussian distribution (using the same parameters we had in the previous examples).

In [ ]:
number_particles = 1000

# Sample from the prior Gaussian distribution
samples = multivariate_normal.rvs(np.array([0, 50, 0, 50]),
                                  np.diag([5, 0, 5, 0]),
                                  size=number_particles)

# Create prior particle state.
prior = ParticleState(state_vector=StateVectors(samples.T),
                      weight=np.array([Probability(1/number_particles)]*number_particles),
                      timestamp=start_time)

In [ ]:
# Measurement Model
measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[5, 0 ],  
                          [0, 5]])
    )  #Covariance matrix for Gaussian PDF

# Transition Model
q_x = 50
q_y = 50
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

predictor = ParticlePredictor(transition_model)
resampler = ESSResampler()
updater = ParticleUpdater(measurement_model, resampler)

start_time = min(timestamps)

track = Track()

for det in dets:
    prediction = predictor.predict(prior, timestamp=det.timestamp)
    # print(det.timestamp)
    hypothesis = SingleHypothesis(prediction, det)   # Group a prediction and measurement
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [ ]:
plotter = AnimatedPlotterly(timestamps, tail_length=.5)

plotter.plot_tracks(track, [0, 2], particle=True, plot_history=False)

plotter.plot_measurements(dets,
                          mapping=[0, 2],
                          measurements_label='Test Data',
                          marker=dict(color='rgba(255, 0, 0, 0.7)',
                                      size=5, 
                                      symbol="cross"))

plotter.fig.update_layout(title={'text': "Test Track - Cartesian - Unscented Kalman", 
                                 'x': 0.5, 
                                 'xanchor': 'center', 
                                 'yanchor':  'top'})

plotter.fig

Changing the starting prior didn't change the results much at all - so not very sensitive to that! However, the first track point is always oddly somewhere between the radar and the actual first observation. Maybe this will clean up in later iterations of the process.

## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

In [ ]:
# rdp = RDPReader(rdp_file,
start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()
timestamps = generate_timestamps(start_time, end_time)

# Detections
meas_polar = CSVReaderPolar(matched_csv)

# Ground Truth
adsb = ADSBTruthReader(adsb_file)
ground_truth = set()
for time, truths in adsb:
    ground_truth.update(truths)

dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]

plotter = AnimatedPlotterly(timestamps, tail_length=0.3, sim_duration=1)

#Animated
plotter.plot_ground_truths(ground_truth, 
                           mapping=[0, 2], 
                           mode='markers', 
                           marker=dict(color='rgba(0, 0, 255, 0.2)',
                                      size=5, 
                                      symbol="square-open")
                            )

plotter.plot_measurements(dets,
                          mapping=[0, 2],
                          measurements_label='Test Data',
                          marker=dict(color='rgba(255, 0, 0, 0.7)',
                                      size=5, 
                                      symbol="cross"),
                          convert_measurements=True
)

plotter.fig.update_layout(title={'text': "Test Track - Polar", 
                                 'x': 0.5, 
                                 'xanchor': 'center', 
                                 'yanchor':  'top'})
# plt.grid()
plotter.fig

### Particle Filtering Again

#### Reset the Prior Samples

In [ ]:
number_particles = 1000

# Sample from the prior Gaussian distribution
samples = multivariate_normal.rvs(np.array([0, 1, 0, 1]),
                                  np.diag([1.5, 0.5, 1.5, 0.5]),
                                  size=number_particles)

# Create prior particle state.
prior = ParticleState(state_vector=StateVectors(samples.T),
                      weight=np.array([Probability(1/number_particles)]*number_particles),
                      timestamp=start_time)

In [ ]:
predictor = ParticlePredictor(transition_model)
resampler = ESSResampler()
updater = ParticleUpdater(measurement_model, resampler)

start_time = min(timestamps)

track = Track()

dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]
for det in dets:
    prediction = predictor.predict(prior, timestamp=det.timestamp)
    # print(det.timestamp)
    hypothesis = SingleHypothesis(prediction, det)   # Group a prediction and measurement
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [ ]:
plotter = AnimatedPlotterly(timestamps, tail_length=0.3, sim_duration=1)

plotter.plot_tracks(track, [0, 2], uncertainty=True, particle=True)

plotter.plot_measurements(dets,
                          mapping=[0, 2],
                          measurements_label='Test Data',
                          marker=dict(color='rgba(255, 0, 0, 0.7)',
                                      size=5, 
                                      symbol="cross"),
                          convert_measurements=True)

plotter.fig